# Titanic - Machine Learning from Disaster


Kaggle link: https://www.kaggle.com/c/titanic

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):    # ''
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


Import all the needed library and init Weights and Biases

In [5]:
import numpy as np

import torch
import torch.nn as nn
torch.manual_seed(0)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats
import pandas as pd

#import wandb
#wandb.init(project='titanic_kaggle')

We first need to read the datasets

In [7]:
titanic_training_data = pd.read_csv('/kaggle/input/titanic/train.csv')    #/kaggle/input/titanic/train.csv './titanic/train.csv'
titanic_test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
titanic_training_data.shape

(891, 12)

Dataframe needs to be cleaned, knowing if some informations are unknown can be very important to determine if someone survived

In [8]:
def clean_titanic(df, train=True):
    df["Cabin"] = df["Cabin"].apply(lambda x: pd.isna(x)).astype(bool)
    df["Embarked"] = df["Embarked"].apply(lambda x: pd.isna(x)).astype(bool)
    df["AgeNan"] = df["Age"].apply(lambda x: pd.isna(x)).astype(bool)
    df = pd.concat([df, pd.get_dummies(df['Sex'], dtype='bool', prefix='sex_'), pd.get_dummies(df['Pclass'], dtype='bool', prefix='pclass_')], axis=1)
    df = df.drop(['PassengerId', 'Name','Ticket','Sex','Pclass'], axis=1)
    if train:
        df = df.drop(['Survived'], axis=1)
    numeric_features = df.dtypes[(df.dtypes != 'object') & (df.dtypes != 'bool')].index
    df[numeric_features] = df[numeric_features].apply(lambda x: (x - x.mean()) / (x.std())) #normalization
    df["Age"] = df["Age"].fillna(df["Age"].mean())
    df["Fare"] = df["Fare"].fillna(df["Fare"].mean())
    return df

y_data = torch.tensor(titanic_training_data["Survived"].values, dtype=torch.float32)
X_data = clean_titanic(titanic_training_data)
X_data.head()

,Age,SibSp,Parch,Fare,Cabin,Embarked,AgeNan,sex__female,sex__male,pclass__1,pclass__2,pclass__3
0,-0.530005,0.432550,-0.473408,-0.502163,True,False,False,False,True,False,False,True
1,0.571430,0.432550,-0.473408,0.786404,False,False,False,True,False,True,False,False
2,-0.254646,-0.474279,-0.473408,-0.488580,True,False,False,True,False,False,False,True
3,0.364911,0.432550,-0.473408,0.420494,False,False,False,True,False,True,False,False
4,0.364911,-0.474279,-0.473408,-0.486064,True,False,False,False,True,False,False,True


We then transform the data from numpy (pandas representation) into torch's `Tensor`

In [10]:
X_data = torch.tensor(X_data.astype('float').values, dtype=torch.float32)
X_data.shape

torch.Size([891, 12])

Create a `TensorDataset` to get tuple of data and label

In [13]:
dataset = torch.utils.data.TensorDataset(X_data, y_data)

We then split between the training and validation set

In [14]:
training_size = int(0.7 * len(dataset))
validation_size = len(dataset) - training_size
train, val = torch.utils.data.random_split(dataset, [training_size, validation_size], generator=torch.Generator().manual_seed(0))
data_loader_train = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
data_loader_val = torch.utils.data.DataLoader(val, batch_size=32, shuffle=True)

Layer initialization using Xavier Uniform on the weight and a constant 0 value on the bias

In [26]:
import torch.nn.functional as F    

def init_my_layer(m):
    torch.nn.init.xavier_normal_(m.weight)
    torch.nn.init.constant_(m.bias, 0)
    return m

class MyNetwork(nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__() 
        self.ln = init_my_layer(nn.Linear(12, 1))

    def forward(self, x):
        x = self.ln(x)
        return F.sigmoid(x) # dim0 is batch, dim1 is input

Create the LinearModel with one Linear layer and Sigmoid applied to the output

In [27]:
#TODO Xavier Uniform to the weight and set the bias to 0
net = MyNetwork() 
print(list(net.parameters()))

[Parameter containing:
tensor([[ 0.4550, -0.0861,  0.7817, -0.1395,  0.0778,  0.6147, -0.4557,  0.1642,
          0.3351,  0.4667,  0.4441,  0.1252]], requires_grad=True), Parameter containing:
tensor([0.], requires_grad=True)]


Initialize the network (call it `net`, it would makes things easier later), the loss, the optimizer and write the training loop

Don't forget to check the validation loss and save your model at the end of each epoch!

In [40]:
num_epochs = 400 # should be more than enought, but can be changed
lr = 3e-3 # e.q to 0.003, you can change it if needed

criterion = nn.BCELoss()    #binary cross entropy loss

optimizer = torch.optim.SGD(net.parameters(), lr)
for epoch in range(num_epochs):
    training_loss = 0
    #TRAINING LOOP
    for X,y in data_loader_train:
        y_pred=net(X)
        #print(y_pred)
        #print(y)
        training_loss=criterion(y_pred,y.reshape(-1, 1))
        optimizer.zero_grad()
        training_loss.backward()
        optimizer.step()
    with torch.no_grad():
        training_loss = criterion(net(X), y.reshape(-1, 1)) 
        print(f'epoch {epoch + 1}, loss {training_loss.item()}')
        #wandb.log({'loss': training_loss.item()}, step=epoch)
    validation_loss = 0
    #VALIDATION LOOP
    for X,y in data_loader_val:
        y_pred=net(X)
        validation_loss=criterion(y_pred,y.reshape(-1, 1))
        optimizer.zero_grad()
        validation_loss.backward()
        optimizer.step()
    with torch.no_grad():
        validation_loss = criterion(net(X), y.reshape(-1, 1)) 
        print(f'epoch {epoch + 1}, loss {validation_loss.item()}')
        #wandb.log({'loss': validation_loss.item()}, step=epoch)
    #wandb.log({'training_loss': training_loss, 'validation_loss': validation_loss})

    # SAVE THE MODEL

epoch 1, loss 0.6451801061630249
epoch 1, loss 0.8304274678230286
epoch 2, loss 0.8780146241188049
epoch 2, loss 0.8694605827331543
epoch 3, loss 0.6734204888343811
epoch 3, loss 0.6706462502479553
epoch 4, loss 0.6563606858253479
epoch 4, loss 0.9786168932914734
epoch 5, loss 0.9601278901100159
epoch 5, loss 0.7496321797370911
epoch 6, loss 0.7228094935417175
epoch 6, loss 0.8860313296318054
epoch 7, loss 0.5008189678192139
epoch 7, loss 0.6323652863502502
epoch 8, loss 0.6718333959579468
epoch 8, loss 0.9441843628883362
epoch 9, loss 0.5553238987922668
epoch 9, loss 0.7147130966186523
epoch 10, loss 0.7528987526893616
epoch 10, loss 0.6519830822944641
epoch 11, loss 1.1612244844436646
epoch 11, loss 0.5732781291007996
epoch 12, loss 0.42038917541503906
epoch 12, loss 0.4850175082683563
epoch 13, loss 0.5414019227027893
epoch 13, loss 0.8064959645271301
epoch 14, loss 0.5226361155509949
epoch 14, loss 0.7703172564506531
epoch 15, loss 0.6077698469161987
epoch 15, loss 0.87427973747253

This loop computes the prediction on the test dataset and create a submission file

You then just have to click the submit button to get your score, lucky you!

In [41]:
titanic_test_data_cleaned = clean_titanic(titanic_test_data, train=False)
titanic_data_tensor = torch.tensor(titanic_test_data_cleaned.astype('float').values, dtype=torch.float32)

with torch.no_grad():
    net.eval()
    test_pred = torch.LongTensor()
    for i, data in enumerate(titanic_data_tensor):
        output = net(data)
        predicted = torch.ge(output, 0.5)
        test_pred = torch.cat((test_pred, predicted), dim=0)
    out_df = pd.DataFrame(np.c_[titanic_test_data['PassengerId'].values, test_pred.numpy()], columns=['PassengerId', 'Survived'])
    out_df.to_csv('submission.csv', index=False)